<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Вводные-данные" data-toc-modified-id="Вводные-данные-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Вводные данные</a></span><ul class="toc-item"><li><span><a href="#Планы-исследования" data-toc-modified-id="Планы-исследования-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Планы исследования</a></span></li></ul></li><li><span><a href="#Загрузка-и-подготовка-данных-к-анализу" data-toc-modified-id="Загрузка-и-подготовка-данных-к-анализу-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и подготовка данных к анализу</a></span><ul class="toc-item"><li><span><a href="#Промежуточные-выводы" data-toc-modified-id="Промежуточные-выводы-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Промежуточные выводы</a></span></li></ul></li><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ данных</a></span><ul class="toc-item"><li><span><a href="#Промежуточные-выводы" data-toc-modified-id="Промежуточные-выводы-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Промежуточные выводы</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Анализ базы данных книг

Необходимо проанализировать базу данных сервиса для чтения книг по подписке. Используем для этого язык запросов SQL.

## Вводные данные

Необходимо проанализировать базу данных сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Нам предоставлено для анализа несколько таблиц:

Таблица books
Содержит данные о книгах:
•	book_id — идентификатор книги;
•	author_id — идентификатор автора;
•	title — название книги;
•	num_pages — количество страниц;
•	publication_date — дата публикации книги;
•	publisher_id — идентификатор издателя.

Таблица authors
Содержит данные об авторах:
•	author_id — идентификатор автора;
•	author — имя автора.

Таблица publishers
Содержит данные об издательствах:
•	publisher_id — идентификатор издательства;
•	publisher — название издательства;

Таблица ratings
Содержит данные о пользовательских оценках книг:
•	rating_id — идентификатор оценки;
•	book_id — идентификатор книги;
•	username — имя пользователя, оставившего оценку;
•	rating — оценка книги.

Таблица reviews
Содержит данные о пользовательских обзорах на книги:
•	review_id — идентификатор обзора;
•	book_id — идентификатор книги;
•	username — имя пользователя, написавшего обзор;
•	text — текст обзора


### Планы исследования

Исследование будет проводить в несколько этапов:

1. Исследуем предоставленные нам данные


2. Ответим на заданные нам вопросы:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

3. Сформируем общие выводы

## Загрузка и подготовка данных к анализу

Загрузим необходимые библиотеки и подключимся к базе данных предоставленной заказчиком

In [1]:
# импортируем необходимые нам библиотеки
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display, HTML
import warnings

#введем некотрые настройки
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 400)



In [2]:
# устанавливаем параметры согласно выданным данным

db_config = {'user': 'praktikum_student', # имя пользователя 
             'pwd': 'Sdf4$2;d-d30pp', # пароль 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
             'port': 6432, # порт подключения 
             'db': 'data-analyst-final-project-db'} # название базы данных 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'], 
                                                         db_config['db']) 

# сохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Проверим полученные данные - выведем первые 5 строк каждой таблицы

In [3]:
# сделаем список таблиц
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for t in tables:
    print('\033[1m' + f'Таблица {t}' + '\033[0m')
    
    query = f'SELECT * FROM {t} LIMIT 5'
    
    #df = sql_request(query)
    display(pd.io.sql.read_sql(query, con = engine))
    
    print('=' * 45)
    print()

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Таблица publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


### Промежуточные выводы

Все таблицы подключились. Везде есть обещанные нам данные. Мы можем начать исследования

## Анализ данных

Для выполнения задания, нам необходимо ответить на следующие вопросы:

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Рассчитаем количество относительно свежих книг

In [4]:
query = '''
SELECT
    COUNT(DISTINCT book_id) AS fresh_books
FROM
    books
WHERE
    publication_date >= '2000-01-01'
ORDER BY 
    fresh_books
'''

x = pd.io.sql.read_sql(query, con = engine)['fresh_books'][0]

print()
print('\033[1m' + 'Количество книг, которые вышли после 1 января 2000 года:' + '\033[0m',x)
print()


Количество книг, которые вышли после 1 января 2000 года: 821



Посчитаем количество обзоров и среднюю оценку для каждой книги

In [5]:
query = '''
SELECT
    books.book_id AS book_id,
    books.title,
    COUNT(DISTINCT reviews.review_id) AS count_of_reviews,
    AVG(ratings.rating) AS mean_rating
FROM
    books
LEFT JOIN reviews ON reviews.book_id = books.book_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    books.book_id
'''

x = pd.io.sql.read_sql(query, con = engine)

print()
print('\033[1m' + 'Количество обзоров и средняя оценка книги:' + '\033[0m')
display(x[['title','count_of_reviews','mean_rating']].round(2))
print()



Количество обзоров и средняя оценка книги:


,title,count_of_reviews,mean_rating
0,'Salem's Lot,2,3.67
1,1 000 Places to See Before You Die,1,2.50
2,13 Little Blue Envelopes (Little Blue Envelope #1),3,4.67
3,1491: New Revelations of the Americas Before Columbus,2,4.50
4,1776,4,4.00
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,Xenocide (Ender's Saga #3),3,3.40
997,Year of Wonders,4,3.20
998,You Suck (A Love Story #2),2,4.50


Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [6]:
query = '''
SELECT
    publishers.publisher AS publisher,
    COUNT(DISTINCT books.book_id) AS fat_books
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages >= 50
GROUP BY
    publishers.publisher_id
ORDER BY
    fat_books DESC
LIMIT
    1
'''

x = pd.io.sql.read_sql(query, con = engine)['publisher'][0]

print()
print('\033[1m' + 'Издательство, выпускающее наибольшее количество книг:' + '\033[0m',x)
print()


Издательство, выпускающее наибольшее количество книг: Penguin Books



Определите автора с самой высокой средней оценкой книг, при этом будем учитывать только книги с 50 и более оценками

In [11]:
query = '''
SELECT
    authors.author AS author,
    SUM(subquery.rating_sum) / SUM(subquery.ratings_cnt) AS rating_mean
FROM
    authors
INNER JOIN
    (
    SELECT
        books.author_id AS author_id,
        SUM(ratings.rating) AS rating_sum,
        COUNT(DISTINCT ratings.rating_id) AS ratings_cnt
    FROM
        books
    INNER JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        books.book_id
    ) AS subquery ON subquery.author_id = authors.author_id
WHERE
    subquery.ratings_cnt >= 50
GROUP BY
    authors.author_id
ORDER BY
    rating_mean DESC
LIMIT
    1
'''

x = pd.io.sql.read_sql(query, con = engine)['author'][0]

print()
print('\033[1m' + 'Издательство, выпускающее наибольшее количество книг:' + '\033[0m',x)
print()


Издательство, выпускающее наибольшее количество книг: J.K. Rowling/Mary GrandPré



Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [8]:
query = '''
SELECT
    AVG(subquery1.reviews_cnt) AS mean_reviews
FROM
    (SELECT
        username,
        COUNT(DISTINCT review_id) as reviews_cnt
    FROM 
        reviews
    GROUP BY
        username
    ) AS subquery1

INNER JOIN
    (
    SELECT
        username AS username,
        COUNT(DISTINCT rating_id) AS ratings_cnt
    FROM
        ratings
    GROUP BY
        username
    ) AS subquery2 ON subquery2.username = subquery1.username

WHERE
    subquery2.ratings_cnt > 50
'''
x = pd.io.sql.read_sql(query, con = engine)['mean_reviews'][0]

print()
print('\033[1m' + 'Среднее количество обзоров от самых активных пользователей:' + '\033[0m','{:.0f}'.format(x))
print()


Среднее количество обзоров от самых активных пользователей: 24



### Промежуточные выводы

Была приобретена достаточно актуальная подборка книг - из 1000 книг больше 80% относительно свежие.
Для части книг есть обзоры, для всех книг есть оценки от тех читателей, которые не стали писать отчет.
Если будет принято решение работать в сторону увеличения больших, многостраничных книг, то лучший кандидат для заключения специального договора - издательство "Penguin Books"
Наиболее высокие оценки, как уавтора у J.K. Rowling/Mary GrandPré. В целом это неудивительно, так как её книги о Гарри Поттере очень популярны. При этом возможно стоит дополнительно запросить и исследовать данные о среднем возрасте наших читателей и затем откорректировать планы на приобретение более молодежного контента.
Что же касается активности пользователей, то самые активные пользователи ( написавшие более 50 отзывов) пишут в среднем по 24 отзыва.

## Выводы

Мы получили от заказчика пять таблиц с данными. Все таблицы рабочие и корректно заполненные. После анализа базы можно сказать, что:
Была приобретена достаточно актуальная подборка книг - из 1000 книг больше 80% относительно свежие.
Для части книг есть обзоры, для всех книг есть оценки от тех читателей, которые не стали писать отчет.
Если будет принято решение работать в сторону увеличения больших, многостраничных книг, то лучший кандидат для заключения специального договора - издательство "Penguin Books"
Наиболее высокие оценки, как уавтора у J.K. Rowling/Mary GrandPré. В целом это неудивительно, так как её книги о Гарри Поттере очень популярны. При этом возможно стоит дополнительно запросить и исследовать данные о среднем возрасте наших читателей и затем откорректировать планы на приобретение более молодежного контента.
Что же касается активности пользователей, то самые активные пользователи ( написавшие более 50 отзывов) пишут в среднем по 24 отзыва.

**Рекомендации**

Я рекомендую запросить больше данных для определения того, кто наши пользователи. Затем возможно скорректировать планы продвижения сервиса, наполнение сервиса.